Problems:
1.should achieve model save then keep training (online learning)
2.early stopping not work
3.any like Alexnet for word?
4.dont know how to adjust model, should consider using AWS?
5.textgenrnn

**This is a NLP problem where the task is to classify Fake News in an article. This notebook consists of various stages needed for identifying fake news such as data preprocessing, model experimentation, and evaluation of results.**


In [288]:
# Importing Libraries
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
import tensorflow
#rom tensorflow.keras.layers import Embedding,Dropout
#rom tensorflow.keras.preprocessing.sequence import pad_sequences
#rom tensorflow.keras.models import Sequential
#rom tensorflow.keras.preprocessing.text import one_hot
#rom tensorflow.keras.layers import LSTM,Bidirectional,GRU
#rom tensorflow.keras.layers import Dense

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ouyan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [289]:
# Reading data from csv
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [290]:
train = train[1:100]

In [291]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [292]:
# Displaying rows and columns in dataset
print("There are {} number of rows and {} number of columns for training.".format(train.shape[0],train.shape[1]))
print("There are {} number of rows and {} number of columns for testing.".format(test.shape[0],test.shape[1]))

There are 99 number of rows and 5 number of columns for training.
There are 5200 number of rows and 4 number of columns for testing.


**Checking Null Values**

In [293]:
# Checking the null values in training data.
train.isnull().sum()

id         0
title      1
author    11
text       0
label      0
dtype: int64

In [294]:
# Checking the null values in testing data.
test.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [295]:
# Handling nan values in dataset using empty spaces
def handle_nan(train_data,test_data):
    '''Input: Data to the function containing Nan values.
       Output : Cleaned data containing no Nan values.
       Function: Cleaning Nan values.
     '''
    train = train_data.fillna(" ")
    test  = test_data.fillna(" ")
    return train,test

train,test = handle_nan(train,test)


In [296]:
# Creating a variable "merged" by merging columns "title" and "author"
train["merged"] = train["title"]+" "+train["text"]
test["merged"]  = test["title"]+" "+test["text"]

In [297]:
train.head()

,id,title,author,text,label,merged
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Why the Trut...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0,Jackie Mason: Hollywood Would Love Trump if He...


In [298]:
test.head()

,id,title,author,text,merged
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...","Specter of Trump Loosens Tongues, if Not Purse..."
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,#NoDAPL: Native American Leaders Vow to Stay A...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...","Tim Tebow Will Attempt Another Comeback, This ..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,Keiser Report: Meme Wars (E995) 42 mins ago 1 ...


In [299]:
# Seperating Independent and dependent features
X = train.drop(columns=['label'],axis=1)
y = train['label']

In [300]:
X.head()

,id,title,author,text,merged
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Why the Truth Might Get You Fired Why the Trut...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Iranian woman jailed for fictional unpublished...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",Jackie Mason: Hollywood Would Love Trump if He...


In [301]:
y.head()

1    0
2    1
3    1
4    1
5    0
Name: label, dtype: int64

In [302]:
# Reset Index
messages = X.copy()
messages.reset_index(inplace=True)
messages_test = test.copy()
messages_test.reset_index(inplace=True)

In [303]:
X.head()

,id,title,author,text,merged
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Why the Truth Might Get You Fired Why the Trut...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Iranian woman jailed for fictional unpublished...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",Jackie Mason: Hollywood Would Love Trump if He...


In [304]:
messages.head()

,index,id,title,author,text,merged
0,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
1,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Why the Truth Might Get You Fired Why the Trut...
2,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,15 Civilians Killed In Single US Airstrike Hav...
3,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Iranian woman jailed for fictional unpublished...
4,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",Jackie Mason: Hollywood Would Love Trump if He...


In [305]:
messages_test.head()

,index,id,title,author,text,merged
0,0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...","Specter of Trump Loosens Tongues, if Not Purse..."
1,1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...,Russian warships ready to strike terrorists ne...
2,2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,#NoDAPL: Native American Leaders Vow to Stay A...
3,3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...","Tim Tebow Will Attempt Another Comeback, This ..."
4,4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,Keiser Report: Meme Wars (E995) 42 mins ago 1 ...


# Data Pre-processing
**In Data Pre-processing following steps are followed:** 
**1. Firstly, all the sequences except english characters are removed from the string.**
**2. Next, to avoid false predictions or ambiguity with upper and lowercase, all the characters in strings are converted    to lowercase.**
**3. Next, all the sentences are tokenized into words.**
**4. To facilitate fast processing, stemming is applied to the tokenized words.**
**5. Next, words are joined together and stored in the corpus.**

**Note: In this tutorial, we have used "merged" column for classification task. Also, the loop inside the function runs over all the examples in the merged column.**

In [306]:
from nltk.stem.porter import PorterStemmer

In [307]:
def normalize(data):
    normalized = []
    for i in data:
        i = i.lower()
        # get rid of urls
        i = re.sub('https?://\S+|www\.\S+', '', i)
        # get rid of non words and extra spaces
        i = re.sub('\\W', ' ', i)
        i = re.sub('\n', '', i)
        i = re.sub(' +', ' ', i)
        i = re.sub('^ ', '', i)
        i = re.sub(' $', '', i)
        normalized.append(i)
    return normalized

In [308]:
messages["merged"] = normalize(messages["merged"])

In [309]:
# Performing data preprocessing on column 'title'
ps = PorterStemmer()
def perform_preprocess(data):
    '''Input: Data to be processed
       Output: Preprocessed data
    '''
    corpus = []
    for i in range(0,len(data)):
        review = re.sub('[^a-zA-Z]',' ',data['merged'][i])
        review = review.lower()
        review = review.split()
        review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
    return corpus

In [310]:
train_corpus = perform_preprocess(messages)
#test_corpus  = perform_preprocess(messages_test)

In [311]:
messages["merged"][0]

'flynn hillary clinton big woman on campus breitbart ever get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination hillary clinton remains the big woman on campus in leafy liberal wellesley massachusetts everywhere else votes her most likely to don her inauguration dress for the remainder of her days the way miss havisham forever wore that wedding dress speaking of great expectations hillary rodham overflowed with them 48 years ago when she first addressed a wellesley graduating class the president of the college informed those gathered in 1969 that the students needed no debate so far as i could ascertain as to who their spokesman was to be kind of the like the democratic primaries in 2016 minus the terms unknown then even at a seven sisters school i am very glad that miss adams made it clear that what i am speaking for today is all of us the 400 of us miss rodham told her classmates after appointing herself edger bergen to 

In [312]:
train_corpus[0]

'flynn hillari clinton big woman campu breitbart ever get feel life circl roundabout rather head straight line toward intend destin hillari clinton remain big woman campu leafi liber wellesley massachusett everywher els vote like inaugur dress remaind day way miss havisham forev wore wed dress speak great expect hillari rodham overflow year ago first address wellesley graduat class presid colleg inform gather student need debat far could ascertain spokesman kind like democrat primari minu term unknown even seven sister school glad miss adam made clear speak today us us miss rodham told classmat appoint edger bergen charli mccarthi mortim snerd attend bespectacl granni glass award matronli wisdom least john lennon wisdom took issu previou speaker despit becom first win elect seat u senat sinc reconstruct edward brook came critic call empathi goal protestor critic tactic though clinton senior thesi saul alinski lament black power demagogu elitist arrog repress intoler within new left sim

**Below code converts the pre-processed words to one-hot vectors in the range of vocabulary size=5000. This is done to obtain numerical feature matrix**

In [313]:
# Converting to one-hot repr.
vocab_size = 5000
one_hot_train = [one_hot(word,vocab_size) for word in train_corpus]
#one_hot_test  = [one_hot(word,vocab_size) for word in test_corpus]

In [314]:
len(one_hot_train)

99

In [315]:
len(one_hot_train[0])

374

**Below code creates an embedding layer which applies "pre" padding to the one-hot encoded features with sentence length = 20. Padding is applied so that the length of every sequence in the dataset should be same.**

In [316]:
# Embedding Representation 
sent_length = 500
embedd_docs_train = pad_sequences(one_hot_train,padding='pre',maxlen=sent_length)
#embedd_docs_test  = pad_sequences(one_hot_test,padding='pre',maxlen=sent_length)

In [317]:
len(embedd_docs_train[0])

500

In [318]:
# Converting Embedding repr. to array
x_final = np.array(embedd_docs_train)
y_final = np.array(y)

#x_test_final = np.array(embedd_docs_test)

In [319]:
# Dimensions of prev. array repr.
x_final.shape, y_final.shape#, x_test_final.shape

((99, 500), (99,))

**Dividing the dataset into training and testing data.**

In [320]:
from sklearn.model_selection import train_test_split

In [321]:
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.1, random_state=42, stratify = y_final)

In [322]:
x_train.shape

(89, 500)

In [323]:
y_train.shape

(89,)

**LSTM**

**In this model, 1.) The value for embedding feature vectors = 40 which are target feature vectors for the embedding layer. 2.) Single LSTM Layer with 100 nodes are used. 3.)Dense Layer with 1 neuron and sigmoid activation function is used since, this is a binary classification problem. 4) Dropout technique is used to avoid overfiiting and adam optimizer is used for optimizing the loss function.**

In [324]:
# Creating the LSTM Model for prediction
embedding_feature_vector = 40
def create_model():
    model = Sequential()
    model.add(Embedding(vocab_size,embedding_feature_vector,input_length=sent_length))
    model.add(Dropout(0.3))
    model.add(LSTM(100))
    model.add(Dropout(0.3))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

    return model

In [325]:
model = create_model()
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 500, 40)           200000    
                                                                 
 dropout_8 (Dropout)         (None, 500, 40)           0         
                                                                 
 lstm_4 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_9 (Dropout)         (None, 100)               0         
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [335]:
# Training the model
model.fit(x_train,y_train,validation_split=0.1,epochs=100,batch_size=64)

Epoch 1/100
2/2 [==============================] - 0s 92ms/step - loss: 3.4001e-04 - accuracy: 1.0000 - val_loss: 1.5545 - val_accuracy: 0.7778
Epoch 2/100
2/2 [==============================] - 0s 71ms/step - loss: 2.2929e-04 - accuracy: 1.0000 - val_loss: 1.5502 - val_accuracy: 0.7778
Epoch 3/100
2/2 [==============================] - 0s 62ms/step - loss: 3.3749e-04 - accuracy: 1.0000 - val_loss: 2.1648 - val_accuracy: 0.6667
Epoch 4/100
2/2 [==============================] - 0s 74ms/step - loss: 3.0621e-04 - accuracy: 1.0000 - val_loss: 2.0641 - val_accuracy: 0.6667
Epoch 5/100
2/2 [==============================] - 0s 72ms/step - loss: 3.6601e-04 - accuracy: 1.0000 - val_loss: 2.0631 - val_accuracy: 0.6667
Epoch 6/100
2/2 [==============================] - 0s 71ms/step - loss: 2.6709e-04 - accuracy: 1.0000 - val_loss: 2.0978 - val_accuracy: 0.5556
Epoch 7/100
2/2 [==============================] - 0s 70ms/step - loss: 3.1608e-04 - accuracy: 1.0000 - val_loss: 2.1442 - val_accuracy:

In [286]:
model.save('ML_LSTM_v2.model')

INFO:tensorflow:Assets written to: ML_LSTM_v2.model\assets


INFO:tensorflow:Assets written to: ML_LSTM_v2.model\assets


In [ ]:
#model.load_weights('ML_LSTM_v2.model')

In [334]:
model.evaluate(x_test, y_test)

1/1 [==============================] - 0s 33ms/step - loss: 1.5595 - accuracy: 0.7000


[1.5595232248306274, 0.699999988079071]

In [ ]:
x_test_final.shape

In [ ]:
# Making Predictions on test data
predictions_test = pd.DataFrame(model.predict(x_test_final))
test_id = pd.DataFrame(test["id"])

submission = pd.concat([test_id, predictions_test],axis=1)
submission.columns = ["id","label"]
submission.to_csv("ML_LSTM_v2_Submission.csv",index=False)

In [ ]:
predictions_test

In [ ]:
submission.shape

In [ ]:
submission.head()